<a href="https://colab.research.google.com/github/ruchikaa123/Front-End-Checklist/blob/master/chest_x_ray.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
!wget http://www.dropbox.com/s/tlxserrdhe240lu/archive.zip

URL transformed to HTTPS due to an HSTS policy
--2021-07-04 07:31:00--  https://www.dropbox.com/s/tlxserrdhe240lu/archive.zip
Resolving www.dropbox.com (www.dropbox.com)... 162.125.2.18, 2620:100:6017:18::a27d:212
Connecting to www.dropbox.com (www.dropbox.com)|162.125.2.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/tlxserrdhe240lu/archive.zip [following]
URL transformed to HTTPS due to an HSTS policy
--2021-07-04 07:31:00--  https://www.dropbox.com/s/raw/tlxserrdhe240lu/archive.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucd13ebb73420c72de7bc2da3631.dl.dropboxusercontent.com/cd/0/inline/BRocnr_IofZSvxctHr1VueqsYX29Q8FDf7rnP_jIpEChL7S1d93fkAg0z_TaVG90b4AMrIegrPGLV3uyhjwQ4lMKcgnvLfUSWnc0BpjHYLnmHiNJfIT4BXkwx-m5KGRcHULchqVW9fUMS5TxJJmYSu9f/file# [following]
--2021-07-04 07:31:00--  https://ucd13ebb73420c72de7bc2da3631.dl.dropboxusercontent.com/cd/0/inline

In [ ]:
!unzip -q "archive.zip"

replace chest_xray/__MACOSX/._chest_xray? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
import tensorflow as tf   #used to build deep learning model and data processing pipelines
import numpy as np        #for fast numerical operations
import matplotlib.pyplot as plt    #for plotting model training and evaluation matrix
from pathlib import Path            # used to play around with path and directories


tf.random.set_seed(4)

In [ ]:
train_path = Path("chest_xray/train/")
validation_path = Path("chest_xray/test")
test_path = Path("chest_xray/val")

In [ ]:
#collecting all the paths inside "normal" and "pneumonia" folder of the above paths

train_image_paths = train_path.glob("*/*")
val_image_paths = validation_path.glob("*/*")

#output is a generator object
print(train_image_paths)

In [ ]:
#convert gnerator object to list of element 

train_image_paths = list(train_image_paths)
val_image_paths = list(val_image_paths)

#now the output are "posixpath" objects
print(train_image_paths[:3])

In [ ]:
# Convert Posix paths to normal strings
train_image_paths = list(map(lambda x : str(x) , train_image_paths))
val_image_paths = list(map(lambda x : str(x) , val_image_paths)) 

print(train_image_paths[:3])

In [ ]:
# Collect Length for Training and Validation Datasets
train_dataset_length = len(train_image_paths)
val_dataset_length = len(val_image_paths)

In [ ]:
# Every Image has Label in its path , so lets slice it 
LABELS = {'NORMAL' : 0 , 'PNEUMONIA' : 1}
INV_LABELS = {0 : 'NORMAL', 1 : 'PNEUMONIA'}

def get_label(path : str) -> int:
    return LABELS[path.split("/")[-2]]

train_labels = list(map(lambda x : get_label(x) , train_image_paths))
val_labels = list(map(lambda x : get_label(x) , val_image_paths))

print(train_labels[:3])


In [ ]:
 #Now we have all training, validation image paths and their respective labels 

BATCH_SIZE = 32

# Function used for Transformation
def load_and_transform(image , label , train = True):
    image = tf.io.read_file(image)
    image = tf.io.decode_jpeg(image , channels = 3)
    image = tf.image.resize(image , [224 , 224] , method="nearest")
    if train:
        image = tf.image.random_flip_left_right(image)
    return image , label

# Function used to Create a Tensorflow Data Object
def get_dataset(paths , labels , train = True):
    image_paths = tf.convert_to_tensor(paths)
    labels = tf.convert_to_tensor(labels)

    image_dataset = tf.data.Dataset.from_tensor_slices(image_paths)
    label_dataset = tf.data.Dataset.from_tensor_slices(labels)

    dataset = tf.data.Dataset.zip((image_dataset , label_dataset)).shuffle(1000)

    dataset = dataset.map(lambda image , label : load_and_transform(image , label , train))
    dataset = dataset.repeat()
    dataset = dataset.shuffle(2048)
    dataset = dataset.batch(BATCH_SIZE)

    return dataset

In [ ]:
#function used to create a tensorflow data object

def get_dataset(paths , labels , train = True):
  image_paths = tf.convert_to_tensor(paths)
  labels = tf.convert_to_tensor(labels)

  image_dataset = tf.data.Dataset.from_tensor_slices(image_paths)
  label_dataset = tf.data.Dataset.from_tensor_slices(labels)

  dataset = tf.data.Dataset.zip((image_dataset , label_dataset)).shuffle(1000)

  dataset = dataset.map(lambda image , label : load_and_transform(image , label , train))
  dataset = dataset.repeat()
  dataset = dataset.shuffle(2048)
  dataset = dataset.batch(BATCH_SIZE)

  return dataset

In [ ]:
 #creating train dataset object and verifying it 

%time train_dataset = get_dataset(train_image_paths, train_labels)

image , label = next(iter(train_dataset))
print(image.shape)
print(label.shape)

In [ ]:
#view a sampe train images
print(INV_LABELS[label[0].numpy()])

plt.imshow(image[0].numpy().reshape(224 , 224 , 3))


In [ ]:

%time val_dataset = get_dataset(val_image_paths, val_labels, train = False)

image , label = next(iter(val_dataset))
print(image.shape)
print(label.shape)

In [ ]:
# View a sample Validation Image
print(INV_LABELS[label[0].numpy()])
plt.imshow(image[0].numpy().reshape(224 , 224 , 3))